In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import json

import pandas as pd

from gzreduction.vote_sources.panoptes_exports import export_to_responses
from gzreduction.vote_sources.panoptes_exports import aggregate
from gzreduction.schemas.jwst_cosmos_schema import jwst_cosmos_schema

In [46]:

schema = jwst_cosmos_schema

In [47]:
classification_export_loc = '/home/walml/repos/gz-panoptes-reduction/data/jwst_cosmos/jwst-cosmos-classifications.csv'
aggregated_loc = '/home/walml/repos/gz-panoptes-reduction/data/jwst_cosmos/jwst-cosmos_reduced_votes.csv'

In [48]:
# nrows = 10000
nrows = None
clf = pd.read_csv(classification_export_loc, nrows=nrows)
clf

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
clf['created_at'] = pd.to_datetime(clf['created_at'])
clf = clf[clf['created_at'] > '2025-04-29']  # live on 29th


In [65]:
clf.iloc[-1]

classification_id                                            637536237
user_name                                                    stegs5434
user_id                                                      2870132.0
user_ip                                           dbbf87ebe502b08197be
workflow_id                                                      28504
workflow_name                                              JWST COSMOS
workflow_version                                                   2.1
created_at                                   2025-05-08 13:09:39+00:00
gold_standard                                                      NaN
expert                                                             NaN
metadata             {"source":"api","session":"2485e93d1f3b6f8774f...
annotations          [{'task': 'T0', 'task_label': 'Is the **centra...
subject_data         {"108637672":{"retired":{"id":140563598,"workf...
subject_ids                                                  108637672
id_str

In [66]:
clf.iloc[-1]['metadata']

'{"source":"api","session":"2485e93d1f3b6f8774f6ca1b518d751119d4214697521520b4ee7972628b9c59","viewport":{"width":1300,"height":561},"started_at":"2025-05-08T13:09:32.707Z","user_agent":"Mozilla/5.0 (X11; CrOS x86_64 14541.0.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36","utc_offset":"18000","finished_at":"2025-05-08T13:09:38.639Z","live_project":true,"interventions":{"opt_in":true,"messageShown":false},"user_language":"en","user_group_ids":[],"subject_dimensions":[{"clientWidth":776,"clientHeight":401,"naturalWidth":1559,"naturalHeight":806},null],"subject_selection_state":{"retired":true,"selected_at":"2025-05-08T13:04:39.195Z","already_seen":false,"selection_state":"failover_fallback","finished_workflow":true,"user_has_finished_workflow":true},"workflow_translation_id":"84788"}'

In [67]:
clf.iloc[-1]['subject_data']

'{"108637672":{"retired":{"id":140563598,"workflow_id":28504,"classifications_count":102,"created_at":"2025-04-29T08:25:03.330Z","updated_at":"2025-05-06T10:44:25.452Z","retired_at":"2025-05-06T10:44:25.439Z","subject_id":108637672,"retirement_reason":"classification_count"},"!RA":"150.12609079","!Dec":"2.3760928","!radius":"5.414316910495499e-07","!ned_link":"[Click to search NASA NED](+tab+https://ned.ipac.caltech.edu/conesearch?search_type=Near%20Position%20Search\\u0026in_csys=Equatorial\\u0026in_equinox=J2000\\u0026ra=150.12609079d\\u0026dec=2.3760928d\\u0026radius=0.1)","#id_field":"687565_B8","!desi_link":"[Click to view in DESI](+tab+https://www.legacysurvey.org/viewer?ra=150.12609079\\u0026dec=2.3760928\\u0026layer=ls-dr9\\u0026zoom=15)","COSMOS_info":"Thanks to the COSMOS-Web team for their support! [Learn more about COSMOS-Web here.](+tab+https://cosmos.astro.caltech.edu/page/cosmosweb)","!esasky_link":"[Click to view in ESA Sky](+tab+https://sky.esa.int/esasky/?target=150.1

In [ ]:
clf['created_at'].iloc[0]

Timestamp('2025-04-29 04:07:02+0000', tz='UTC')

In [49]:
def get_jwst_subject_id(subject_data):
    loaded = json.loads(subject_data)
    inner_dict = loaded[list(loaded.keys())[0]]
    # print(inner_dict)
    
    # seems changed?
    status_dict = inner_dict['retired']
    
    try:
        return status_dict['id']
    except KeyError:
        try:
            return status_dict['!id']
        except KeyError:
            return None
        

In [50]:
clf['id_str'] = clf['subject_data'].apply(get_jwst_subject_id)

/tmp/ipykernel_14320/991062309.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clf['id_str'] = clf['subject_data'].apply(get_jwst_subject_id)


In [52]:
clf['id_str'].value_counts(dropna=False)

id_str
140562844    142
140557914    140
140555706    136
140564739    135
140564475    135
            ... 
140557409     40
140559293     40
140559660     40
140557041     40
140559984     40
Name: count, Length: 4709, dtype: int64

In [53]:
workflows = [28504]
clf = clf[clf['workflow_id'].isin(workflows)]

In [55]:
clf

,classification_id,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,metadata,annotations,subject_data,subject_ids,id_str
4,634692912,allisonava08,2864008.0,a5bae168f8cdbe2b5cca,28504,JWST COSMOS,2.1,2025-04-29 04:07:02+00:00,NaN,NaN,"{""source"":""api"",""session"":""a35ee2f9ed710ecdb12...","[{""task"":""T0"",""task_label"":""Is the **central g...","{""108665729"":{""retired"":{""id"":140555828,""workf...",108665729,140555828
5,634692959,allisonava08,2864008.0,2c1738093a8bbf18077d,28504,JWST COSMOS,2.1,2025-04-29 04:07:27+00:00,NaN,NaN,"{""source"":""api"",""session"":""a35ee2f9ed710ecdb12...","[{""task"":""T0"",""task_label"":""Is the **central g...","{""108682329"":{""retired"":{""id"":140555845,""workf...",108682329,140555845
6,634692987,allisonava08,2864008.0,3707d06f4d2d82d83f71,28504,JWST COSMOS,2.1,2025-04-29 04:07:40+00:00,NaN,NaN,"{""source"":""api"",""session"":""a35ee2f9ed710ecdb12...","[{""task"":""T0"",""task_label"":""Is the **central g...","{""108657542"":{""retired"":{""id"":140555850,""workf...",108657542,140555850
7,634693016,allisonava08,2864008.0,7f13b3a8a531f73bdeae,28504,JWST COSMOS,2.1,2025-04-29 04:07:54+00:00,NaN,NaN,"{""source"":""api"",""session"":""a35ee2f9ed710ecdb12...","[{""task"":""T0"",""task_label"":""Is the **central g...","{""108640888"":{""retired"":{""id"":140555856,""workf...",108640888,140555856
8,634693079,allisonava08,2864008.0,a307837a46e6d210be71,28504,JWST COSMOS,2.1,2025-04-29 04:08:22+00:00,NaN,NaN,"{""source"":""api"",""session"":""a35ee2f9ed710ecdb12...","[{""task"":""T0"",""task_label"":""Is the **central g...","{""108639693"":{""retired"":{""id"":140555866,""workf...",108639693,140555866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197259,637535796,stegs5434,2870132.0,82c9b893536f35ae59e1,28504,JWST COSMOS,2.1,2025-05-08 13:08:08+00:00,NaN,NaN,"{""source"":""api"",""session"":""2485e93d1f3b6f8774f...","[{""task"":""T0"",""task_label"":""Is the **central g...","{""108637747"":{""retired"":{""id"":140562793,""workf...",108637747,140562793
197260,637535825,stegs5434,2870132.0,82c9b893536f35ae59e1,28504,JWST COSMOS,2.1,2025-05-08 13:08:14+00:00,NaN,NaN,"{""source"":""api"",""session"":""2485e93d1f3b6f8774f...","[{""task"":""T0"",""task_label"":""Is the **central g...","{""108641891"":{""retired"":{""id"":140557031,""workf...",108641891,140557031
197261,637536012,stegs5434,2870132.0,c187ce8a4be0abeba301,28504,JWST COSMOS,2.1,2025-05-08 13:08:52+00:00,NaN,NaN,"{""source"":""api"",""session"":""2485e93d1f3b6f8774f...","[{""task"":""T0"",""task_label"":""Is the **central g...","{""108637246"":{""retired"":{""id"":140555803,""workf...",108637246,140555803
197262,637536206,stegs5434,2870132.0,8b99c4e025f83127c36b,28504,JWST COSMOS,2.1,2025-05-08 13:09:33+00:00,NaN,NaN,"{""source"":""api"",""session"":""2485e93d1f3b6f8774f...","[{""task"":""T0"",""task_label"":""Is the **central g...","{""108638014"":{""retired"":{""id"":140555824,""workf...",108638014,140555824


In [57]:
clf['annotations'] = clf['annotations'].apply(lambda x: json.loads(x))

In [56]:
clf['annotations'].iloc[0]

'[{"task":"T0","task_label":"Is the **central galaxy** simply smooth and rounded, with no sign of a disk?","value":"![star.png](https://panoptes-uploads.zooniverse.org/production/project_attached_image/cd46dc5a-44b3-426e-82c1-31812ca65a3e.png =60x) Star, Artifact, or Bad Zoom"},{"task":"T19","task_label":"What problem do you see with the image?","value":"![star.png](https://panoptes-uploads.zooniverse.org/production/project_attached_image/cd46dc5a-44b3-426e-82c1-31812ca65a3e.png =60x) Star"}]'

In [58]:
df = export_to_responses.explode_annotations(clf, exclude_tasks=['T10'])
df

,task,task_label,value,id_str,user_id,classification_id,created_at,subject_ids
4,T0,Is the **central galaxy** simply smooth and ro...,![star.png](https://panoptes-uploads.zoonivers...,140555828,2864008.0,634692912,2025-04-29 04:07:02+00:00,108665729
4,T19,What problem do you see with the image?,![star.png](https://panoptes-uploads.zoonivers...,140555828,2864008.0,634692912,2025-04-29 04:07:02+00:00,108665729
5,T0,Is the **central galaxy** simply smooth and ro...,![star.png](https://panoptes-uploads.zoonivers...,140555845,2864008.0,634692959,2025-04-29 04:07:27+00:00,108682329
5,T19,What problem do you see with the image?,![bad_zoom_smaller_lens.png](https://panoptes-...,140555845,2864008.0,634692959,2025-04-29 04:07:27+00:00,108682329
6,T0,Is the **central galaxy** simply smooth and ro...,![star.png](https://panoptes-uploads.zoonivers...,140555850,2864008.0,634692987,2025-04-29 04:07:40+00:00,108657542
...,...,...,...,...,...,...,...,...
197261,T11,Is the galaxy merging or disturbed?,![no.png](https://panoptes-uploads.zooniverse....,140555803,2870132.0,637536012,2025-05-08 13:08:52+00:00,108637246
197262,T0,Is the **central galaxy** simply smooth and ro...,![star.png](https://panoptes-uploads.zoonivers...,140555824,2870132.0,637536206,2025-05-08 13:09:33+00:00,108638014
197262,T19,What problem do you see with the image?,![bad_zoom_smaller_lens.png](https://panoptes-...,140555824,2870132.0,637536206,2025-05-08 13:09:33+00:00,108638014
197263,T0,Is the **central galaxy** simply smooth and ro...,![star.png](https://panoptes-uploads.zoonivers...,140563598,2870132.0,637536237,2025-05-08 13:09:39+00:00,108637672


In [ ]:
# question = schema.get_question_from_raw_name('T0')
# # question.get_answer_names()
# question.get_raw_answer_names()

['smooth', 'features or disk', 'star, artifact, or bad zoom']

In [60]:
cleaned = export_to_responses.clean_exploded_annotations(df, schema)

In [61]:
aggregated = aggregate.responses_to_reduced_votes(cleaned.reset_index(drop=True), schema)

In [62]:
aggregated

question_response,id_str,smooth-or-featured_smooth,smooth-or-featured_featured-or-disk,smooth-or-featured_star-artifact-zoom,how-rounded_round,how-rounded_in-between,how-rounded_cigar-shaped,disk-edge-on_yes,disk-edge-on_no,edge-on-bulge_rounded,...,bulge-size_dominant_fraction,merging_merger_fraction,merging_major-disturbance_fraction,merging_minor-disturbance_fraction,merging_none_fraction,clumps_yes_fraction,clumps_no_fraction,problem_star_fraction,problem_artifact_fraction,problem_bad-zoom_fraction
0,140468011,17,17,6,0,0,0,15,2,9,...,0.000000,0.029412,0.058824,0.235294,0.676471,0.058824,0.941176,0.000000,0.166667,0.833333
1,140468173,25,0,15,0,0,0,0,0,0,...,0.000000,0.000000,0.040000,0.200000,0.760000,0.000000,0.000000,0.133333,0.000000,0.866667
2,140468449,30,6,4,0,0,0,0,6,0,...,0.333333,0.055556,0.055556,0.138889,0.750000,0.000000,1.000000,0.250000,0.000000,0.750000
3,140469172,30,4,6,0,0,0,0,4,0,...,0.250000,0.000000,0.088235,0.147059,0.764706,0.000000,1.000000,0.000000,0.000000,1.000000
4,140508541,16,16,8,0,0,0,14,2,6,...,0.000000,0.000000,0.093750,0.218750,0.687500,0.062500,0.937500,0.000000,0.250000,0.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4704,140564997,5,2,33,0,0,0,0,2,0,...,0.000000,0.571429,0.142857,0.142857,0.142857,0.500000,0.500000,0.060606,0.121212,0.818182
4705,140565001,32,2,7,0,0,0,1,1,1,...,1.000000,0.029412,0.000000,0.205882,0.764706,0.000000,1.000000,0.000000,0.000000,1.000000
4706,140565007,18,9,14,0,0,0,0,9,0,...,0.111111,0.703704,0.037037,0.037037,0.222222,0.666667,0.333333,0.000000,0.000000,1.000000
4707,140565008,21,0,20,0,0,0,0,0,0,...,0.000000,0.047619,0.000000,0.095238,0.857143,0.000000,0.000000,0.350000,0.000000,0.650000


In [83]:
# merge in subject data

final = pd.merge(
    aggregated,
    clf[['id_str', 'subject_ids']].drop_duplicates(),
    on='id_str',
    how='left'
)
final = final.rename(columns={'subject_ids': 'subject_id'})
 # big 4GB csv
subject_df = pd.read_csv('/home/walml/repos/gz-panoptes-reduction/data/euclid/galaxy-zoo-subjects.csv', usecols=['subject_id', 'locations'])
final = final.merge(subject_df, on='subject_id', how='left')

final['subject_id'].isna().mean()

0.0

In [87]:
final['locations']

0       {"0":"https://panoptes-uploads.zooniverse.org/...
1       {"0":"https://panoptes-uploads.zooniverse.org/...
2       {"0":"https://panoptes-uploads.zooniverse.org/...
3       {"0":"https://panoptes-uploads.zooniverse.org/...
4       {"0":"https://panoptes-uploads.zooniverse.org/...
                              ...                        
4705    {"0":"https://panoptes-uploads.zooniverse.org/...
4706    {"0":"https://panoptes-uploads.zooniverse.org/...
4707    {"0":"https://panoptes-uploads.zooniverse.org/...
4708    {"0":"https://panoptes-uploads.zooniverse.org/...
4709    {"0":"https://panoptes-uploads.zooniverse.org/...
Name: locations, Length: 4710, dtype: object

In [89]:
final['url_first_frame'] = final['locations'].apply(lambda x: json.loads(x)["0"])
final['url_second_frame'] = final['locations'].apply(lambda x: json.loads(x)["1"])

In [93]:
final['url_first_frame'].iloc[-1]

'https://panoptes-uploads.zooniverse.org/subject_location/be4a888d-1521-4e44-851b-15c33d42a8b4.png'

In [94]:
final['url_second_frame'].iloc[-1]

'https://panoptes-uploads.zooniverse.org/subject_location/b49ccd32-ed1d-4433-a48a-4df2283eb7c8.png'

In [90]:
final.to_csv(aggregated_loc, index=False)

In [91]:
final['smooth-or-featured_smooth_fraction']

0       0.425000
1       0.625000
2       0.750000
3       0.750000
4       0.400000
          ...   
4705    0.125000
4706    0.780488
4707    0.439024
4708    0.512195
4709    0.675000
Name: smooth-or-featured_smooth_fraction, Length: 4710, dtype: float64